In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

C:\Users\kisho\AppData\Local\Temp\ipykernel_10496\2495388112.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
nestdf = pd.read_csv('./Nest.csv')
partavgdf = pd.read_csv('./Part_Avg.csv')

In [4]:
nestdf.head()

,ixJobSummary,fStrategies,dLengthUsed_Avg,dWidthUsed_Avg,dPartArea_Job,dTrueAreaRectified_Job
0,130032,0,240.00000,95.99000,519.0404,23037.600000
1,145075,8,108.45324,39.26320,18389.9746,21573.573062
2,233110,8,38.70940,35.73610,879.9165,1383.322989
3,286531,8,69.70170,60.55855,5992.2342,9448.707993
4,504531,0,6.80700,38.00690,198.8594,258.712968


In [5]:
joined_nest_partsavg_df = pd.merge(nestdf, partavgdf, on='ixJobSummary', how='inner')
joined_nest_partsavg_df #avg num of parts and nest data in one df

,ixJobSummary,fStrategies,dLengthUsed_Avg,dWidthUsed_Avg,dPartArea_Job,dTrueAreaRectified_Job,dLength_Avg,dWidth_Avg,dArea_Avg,cNested_Avg,fExtShape_Avg,dExtArea_Avg,dExtBoundaryDist_Avg,dExtContainedDist_Avg,dLgIntArea_Avg,dLgIntBoundaryDist_Avg,dLgIntContainedDist_Avg,dLgExtConArea_Avg,dLgExtConBoundaryDist,dLgExtConContainedDist
0,130032,0,240.000000,95.990000,519.0404,23037.600000,11.214286,7.588371,58.148629,1,9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,145075,8,108.453240,39.263200,18389.9746,21573.573062,38.347309,12.843400,475.076800,4,5,496.283591,6.026536,39.581636,14.593373,0.535145,2.846391,12.247745,0.341673,0.728891
2,233110,8,38.709400,35.736100,879.9165,1383.322989,37.702200,20.209600,439.958200,2,0,439.958200,9.161500,36.520700,0.000000,0.000000,0.000000,60.162600,2.761000,12.675600
3,286531,8,69.701700,60.558550,5992.2342,9448.707993,17.103650,18.125000,191.540050,12,0,193.776600,6.589750,20.581050,0.279550,0.000000,0.125000,35.632250,2.375000,4.500000
4,504531,0,6.807000,38.006900,198.8594,258.712968,37.000000,5.800000,198.859400,1,0,198.859400,2.644400,37.148100,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1790931,2583832,0,143.770400,94.319500,3562.2303,13560.352243,52.203125,32.046875,815.453875,1,2,1175.660175,15.350675,52.728425,355.015300,7.125000,15.062500,0.000000,0.000000,0.000000
1790932,548877,4,34.513950,55.300350,2898.6813,4526.656120,23.053571,13.199986,273.049857,4,3,296.410943,6.052186,25.126200,3.782143,0.233057,1.326471,0.000000,0.000000,0.000000
1790933,2420784,8,83.969400,33.398500,1732.8873,2804.452006,83.057750,4.540683,288.814567,1,16,331.347183,2.041967,83.173917,10.515283,0.609483,2.738883,0.000000,0.000000,0.000000
1790934,2302130,8,80.844067,54.065133,7208.6328,14136.765442,43.584450,40.175350,693.526650,5,0,1308.887900,15.457300,51.257850,504.934150,7.507200,22.396550,61.498750,1.939500,9.697650


In [6]:
joined_nest_partsavg_df['CropUtil'] = joined_nest_partsavg_df['dPartArea_Job'] / joined_nest_partsavg_df['dTrueAreaRectified_Job']

joined_nest_partsavg_df.drop('dLengthUsed_Avg', axis=1, inplace=True)
joined_nest_partsavg_df.drop('dWidthUsed_Avg', axis=1, inplace=True)
joined_nest_partsavg_df.drop('dTrueAreaRectified_Job', axis=1, inplace=True)


joined_nest_partsavg_df.drop('ixJobSummary', axis=1, inplace=True)

joined_nest_partsavg_df


,fStrategies,dPartArea_Job,dLength_Avg,dWidth_Avg,dArea_Avg,cNested_Avg,fExtShape_Avg,dExtArea_Avg,dExtBoundaryDist_Avg,dExtContainedDist_Avg,dLgIntArea_Avg,dLgIntBoundaryDist_Avg,dLgIntContainedDist_Avg,dLgExtConArea_Avg,dLgExtConBoundaryDist,dLgExtConContainedDist,CropUtil
0,0,519.0404,11.214286,7.588371,58.148629,1,9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022530
1,8,18389.9746,38.347309,12.843400,475.076800,4,5,496.283591,6.026536,39.581636,14.593373,0.535145,2.846391,12.247745,0.341673,0.728891,0.852431
2,8,879.9165,37.702200,20.209600,439.958200,2,0,439.958200,9.161500,36.520700,0.000000,0.000000,0.000000,60.162600,2.761000,12.675600,0.636089
3,8,5992.2342,17.103650,18.125000,191.540050,12,0,193.776600,6.589750,20.581050,0.279550,0.000000,0.125000,35.632250,2.375000,4.500000,0.634186
4,0,198.8594,37.000000,5.800000,198.859400,1,0,198.859400,2.644400,37.148100,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.768649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1790931,0,3562.2303,52.203125,32.046875,815.453875,1,2,1175.660175,15.350675,52.728425,355.015300,7.125000,15.062500,0.000000,0.000000,0.000000,0.262695
1790932,4,2898.6813,23.053571,13.199986,273.049857,4,3,296.410943,6.052186,25.126200,3.782143,0.233057,1.326471,0.000000,0.000000,0.000000,0.640358
1790933,8,1732.8873,83.057750,4.540683,288.814567,1,16,331.347183,2.041967,83.173917,10.515283,0.609483,2.738883,0.000000,0.000000,0.000000,0.617906
1790934,8,7208.6328,43.584450,40.175350,693.526650,5,0,1308.887900,15.457300,51.257850,504.934150,7.507200,22.396550,61.498750,1.939500,9.697650,0.509921


In [7]:

# Print the DataFrame to verify the changes
joined_nest_partsavg_df

,fStrategies,dPartArea_Job,dLength_Avg,dWidth_Avg,dArea_Avg,cNested_Avg,fExtShape_Avg,dExtArea_Avg,dExtBoundaryDist_Avg,dExtContainedDist_Avg,dLgIntArea_Avg,dLgIntBoundaryDist_Avg,dLgIntContainedDist_Avg,dLgExtConArea_Avg,dLgExtConBoundaryDist,dLgExtConContainedDist,CropUtil
0,0,519.0404,11.214286,7.588371,58.148629,1,9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022530
1,8,18389.9746,38.347309,12.843400,475.076800,4,5,496.283591,6.026536,39.581636,14.593373,0.535145,2.846391,12.247745,0.341673,0.728891,0.852431
2,8,879.9165,37.702200,20.209600,439.958200,2,0,439.958200,9.161500,36.520700,0.000000,0.000000,0.000000,60.162600,2.761000,12.675600,0.636089
3,8,5992.2342,17.103650,18.125000,191.540050,12,0,193.776600,6.589750,20.581050,0.279550,0.000000,0.125000,35.632250,2.375000,4.500000,0.634186
4,0,198.8594,37.000000,5.800000,198.859400,1,0,198.859400,2.644400,37.148100,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.768649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1790931,0,3562.2303,52.203125,32.046875,815.453875,1,2,1175.660175,15.350675,52.728425,355.015300,7.125000,15.062500,0.000000,0.000000,0.000000,0.262695
1790932,4,2898.6813,23.053571,13.199986,273.049857,4,3,296.410943,6.052186,25.126200,3.782143,0.233057,1.326471,0.000000,0.000000,0.000000,0.640358
1790933,8,1732.8873,83.057750,4.540683,288.814567,1,16,331.347183,2.041967,83.173917,10.515283,0.609483,2.738883,0.000000,0.000000,0.000000,0.617906
1790934,8,7208.6328,43.584450,40.175350,693.526650,5,0,1308.887900,15.457300,51.257850,504.934150,7.507200,22.396550,61.498750,1.939500,9.697650,0.509921


In [8]:
# Separate data by strategy
strategy_groups = joined_nest_partsavg_df.groupby('fStrategies')

In [9]:
for strategy, group in strategy_groups:
    print(str(strategy)+" "+str(group.shape))

-2147483648 (520786, 17)
0 (692368, 17)
1 (48044, 17)
2 (10651, 17)
4 (14756, 17)
8 (218856, 17)
16 (20268, 17)
32 (5968, 17)
64 (4433, 17)
128 (16858, 17)
256 (2465, 17)
512 (26634, 17)
1024 (12292, 17)
2048 (6585, 17)
4096 (13020, 17)
8192 (83085, 17)
16384 (93867, 17)


In [39]:
# Placeholder for models and scalers
models = {}
scalers = {}
encoders = {}

In [42]:
import os
import joblib
from keras.models import load_model

# Create directories if they don't exist
os.makedirs('models', exist_ok=True)
os.makedirs('scalers', exist_ok=True)
os.makedirs('encoders', exist_ok=True)


In [10]:
import joblib
from keras.models import load_model
from sklearn.metrics import mean_absolute_error, mean_squared_error

**Getting all the top strategies for each job and seeing if it differs from used strategy and by how much Utilization**

In [11]:
joined_nest_partsavg_df

X = joined_nest_partsavg_df.drop(columns=['CropUtil', 'fStrategies'])
y = joined_nest_partsavg_df['CropUtil']


cols = ['fStrategies', 'CropUtil'] 
comparison_df = joined_nest_partsavg_df[cols] 




In [12]:
X.head()

,dPartArea_Job,dLength_Avg,dWidth_Avg,dArea_Avg,cNested_Avg,fExtShape_Avg,dExtArea_Avg,dExtBoundaryDist_Avg,dExtContainedDist_Avg,dLgIntArea_Avg,dLgIntBoundaryDist_Avg,dLgIntContainedDist_Avg,dLgExtConArea_Avg,dLgExtConBoundaryDist,dLgExtConContainedDist
0,519.0404,11.214286,7.588371,58.148629,1,9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,18389.9746,38.347309,12.843400,475.076800,4,5,496.283591,6.026536,39.581636,14.593373,0.535145,2.846391,12.247745,0.341673,0.728891
2,879.9165,37.702200,20.209600,439.958200,2,0,439.958200,9.161500,36.520700,0.000000,0.000000,0.000000,60.162600,2.761000,12.675600
3,5992.2342,17.103650,18.125000,191.540050,12,0,193.776600,6.589750,20.581050,0.279550,0.000000,0.125000,35.632250,2.375000,4.500000
4,198.8594,37.000000,5.800000,198.859400,1,0,198.859400,2.644400,37.148100,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [11]:
# Placeholder for models and scalers
models = {}
scalers = {}
encoders = {}

In [14]:
for strategy, group in strategy_groups:
    
    # Load the scaler
    scaler_path = f'scalers/{strategy}_scaler.pkl'
    scaler = joblib.load(scaler_path)

    model_path = f'models/{strategy}_rf_model.pkl'
    rf = joblib.load(model_path)
    
    # Load the encoder model
    encoder_path = f'encoders/{strategy}_encoder.h5'
    encoder_model = load_model(encoder_path)

    # Store the models and scalers
    models[strategy] = rf
    scalers[strategy] = scaler
    encoders[strategy] = encoder_model
    # X_scaled = scaler.transform(X)

    


In [18]:
for strategy, group in strategy_groups:
    X_scaled = scalers[strategy].transform(X)

    # Encode the features
    X_scaled_encoded = encoders[strategy].predict(X_scaled)
    
    # Evaluate the model
    y_pred = models[strategy].predict(X_scaled_encoded)

    comparison_df[str(strategy)+" CropUtil"] = y_pred

    
    

55967/55967 ━━━━━━━━━━━━━━━━━━━━ 51s 911us/step


C:\Users\kisho\AppData\Local\Temp\ipykernel_23476\1680777078.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comparison_df[str(strategy)+" CropUtil"] = y_pred


55967/55967 ━━━━━━━━━━━━━━━━━━━━ 102s 2ms/step


C:\Users\kisho\AppData\Local\Temp\ipykernel_23476\1680777078.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comparison_df[str(strategy)+" CropUtil"] = y_pred


55967/55967 ━━━━━━━━━━━━━━━━━━━━ 127s 2ms/step


...

55967/55967 ━━━━━━━━━━━━━━━━━━━━ 42s 759us/step


C:\Users\kisho\AppData\Local\Temp\ipykernel_23476\1680777078.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comparison_df[str(strategy)+" CropUtil"] = y_pred


55967/55967 ━━━━━━━━━━━━━━━━━━━━ 33s 584us/step


C:\Users\kisho\AppData\Local\Temp\ipykernel_23476\1680777078.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comparison_df[str(strategy)+" CropUtil"] = y_pred


55967/55967 ━━━━━━━━━━━━━━━━━━━━ 48s 858us/step
55967/55967 ━━━━━━━━━━━━━━━━━━━━ 73s 1ms/step
55967/55967 ━━━━━━━━━━━━━━━━━━━━ 72s 1ms/step


In [13]:
comparison_df

,fStrategies,CropUtil
0,0,0.022530
1,8,0.852431
2,8,0.636089
3,8,0.634186
4,0,0.768649
...,...,...
1790931,0,0.262695
1790932,4,0.640358
1790933,8,0.617906
1790934,8,0.509921


In [73]:
comparison_df.to_csv('all_strategy_model_prediction_of_whole_data.csv', index=False)

In [14]:
comparison_df = pd.read_csv('./all_strategy_model_prediction_of_whole_data.csv')

In [15]:
only_predicted_crop_util_df = comparison_df.loc[:, ['-2147483648 CropUtil', '0 CropUtil', '1 CropUtil', '2 CropUtil', '4 CropUtil', '8 CropUtil', '16 CropUtil', '32 CropUtil', '64 CropUtil', '128 CropUtil', '256 CropUtil', '512 CropUtil', '1024 CropUtil', '2048 CropUtil', '4096 CropUtil', '8192 CropUtil', '16384 CropUtil']]

In [16]:
only_predicted_crop_util_df

,-2147483648 CropUtil,0 CropUtil,1 CropUtil,2 CropUtil,4 CropUtil,8 CropUtil,16 CropUtil,32 CropUtil,64 CropUtil,128 CropUtil,256 CropUtil,512 CropUtil,1024 CropUtil,2048 CropUtil,4096 CropUtil,8192 CropUtil,16384 CropUtil
0,0.428185,0.023213,0.587571,0.389400,0.607966,0.388689,0.572113,0.593845,0.662942,0.469630,0.534997,0.447242,0.454650,0.656718,0.490652,0.518469,0.543229
1,0.763794,0.754321,0.777328,0.686315,0.756543,0.836341,0.740403,0.746949,0.752615,0.740948,0.742385,0.717565,0.778537,0.711462,0.819340,0.740584,0.812318
2,0.529820,0.574491,0.668039,0.612628,0.546636,0.639049,0.630845,0.639605,0.634645,0.739873,0.590745,0.626445,0.570737,0.598400,0.554239,0.684062,0.703126
3,0.583396,0.646357,0.629869,0.570579,0.464600,0.610241,0.667773,0.616516,0.591157,0.552353,0.595550,0.642489,0.411588,0.366866,0.541052,0.551993,0.709342
4,0.676890,0.695612,0.530754,0.715871,0.570635,0.671794,0.649680,0.572603,0.762631,0.697442,0.769121,0.693819,0.610418,0.702228,0.743961,0.736189,0.718792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1790931,0.612443,0.329363,0.621953,0.673451,0.702255,0.627278,0.573571,0.597149,0.488411,0.517322,0.532240,0.626770,0.678036,0.673735,0.446265,0.568875,0.617616
1790932,0.681094,0.622640,0.640552,0.712032,0.640599,0.745711,0.725636,0.790543,0.743779,0.709275,0.759425,0.662777,0.731539,0.811541,0.735507,0.676120,0.783240
1790933,0.675152,0.641579,0.733993,0.797409,0.745744,0.649768,0.710832,0.562858,0.819938,0.759397,0.711107,0.680330,0.537844,0.701692,0.768657,0.702884,0.789814
1790934,0.500701,0.514418,0.447982,0.621615,0.435785,0.500652,0.522487,0.696437,0.739062,0.628484,0.616472,0.422446,0.661735,0.484772,0.446148,0.541314,0.547318


In [17]:
df = only_predicted_crop_util_df.max(axis=1)

In [18]:
df

0          0.662942
1          0.836341
2          0.739873
3          0.709342
4          0.769121
             ...   
1790931    0.702255
1790932    0.811541
1790933    0.819938
1790934    0.739062
1790935    0.826359
Length: 1790936, dtype: float64

In [19]:
df2 = only_predicted_crop_util_df.idxmax(axis=1)

In [20]:
df2 = df2.str.replace(" CropUtil", '', regex=False)
df2

0             64
1              8
2            128
3          16384
4            256
           ...  
1790931        4
1790932     2048
1790933       64
1790934       64
1790935      128
Length: 1790936, dtype: object

In [21]:
df2 = df2.str.strip()

In [22]:
df2.iloc[1]

'8'

In [27]:
comparison_df['Predicted CropUtil'] = df

In [30]:
comparison_df['Best Strategy Predicted'] = df2
comparison_df['Best Strategy Predicted'] = comparison_df['Best Strategy Predicted'].astype(int)

In [32]:
filtered_comparison_df = comparison_df[comparison_df['fStrategies'] != comparison_df['Best Strategy Predicted']]

In [33]:
filtered_comparison_df

,fStrategies,CropUtil,-2147483648 CropUtil,0 CropUtil,1 CropUtil,2 CropUtil,4 CropUtil,8 CropUtil,16 CropUtil,32 CropUtil,...,512 CropUtil,1024 CropUtil,2048 CropUtil,4096 CropUtil,8192 CropUtil,16384 CropUtil,max croputil predicted,max croputil strategy,Predicted CropUtil,Best Strategy Predicted
0,0,0.022530,0.428185,0.023213,0.587571,0.389400,0.607966,0.388689,0.572113,0.593845,...,0.447242,0.454650,0.656718,0.490652,0.518469,0.543229,0.662942,64,0.662942,64
2,8,0.636089,0.529820,0.574491,0.668039,0.612628,0.546636,0.639049,0.630845,0.639605,...,0.626445,0.570737,0.598400,0.554239,0.684062,0.703126,0.739873,128,0.739873,128
3,8,0.634186,0.583396,0.646357,0.629869,0.570579,0.464600,0.610241,0.667773,0.616516,...,0.642489,0.411588,0.366866,0.541052,0.551993,0.709342,0.709342,16384,0.709342,16384
4,0,0.768649,0.676890,0.695612,0.530754,0.715871,0.570635,0.671794,0.649680,0.572603,...,0.693819,0.610418,0.702228,0.743961,0.736189,0.718792,0.769121,256,0.769121,256
5,128,0.413703,0.274512,0.335525,0.488613,0.447459,0.424580,0.386493,0.417232,0.410657,...,0.438088,0.290763,0.436091,0.469603,0.310771,0.407693,0.606014,256,0.606014,256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1790931,0,0.262695,0.612443,0.329363,0.621953,0.673451,0.702255,0.627278,0.573571,0.597149,...,0.626770,0.678036,0.673735,0.446265,0.568875,0.617616,0.702255,4,0.702255,4
1790932,4,0.640358,0.681094,0.622640,0.640552,0.712032,0.640599,0.745711,0.725636,0.790543,...,0.662777,0.731539,0.811541,0.735507,0.676120,0.783240,0.811541,2048,0.811541,2048
1790933,8,0.617906,0.675152,0.641579,0.733993,0.797409,0.745744,0.649768,0.710832,0.562858,...,0.680330,0.537844,0.701692,0.768657,0.702884,0.789814,0.819938,64,0.819938,64
1790934,8,0.509921,0.500701,0.514418,0.447982,0.621615,0.435785,0.500652,0.522487,0.696437,...,0.422446,0.661735,0.484772,0.446148,0.541314,0.547318,0.739062,64,0.739062,64


In [35]:
cols = ['fStrategies', 'CropUtil', 'Best Strategy Predicted', 'Predicted CropUtil'] 
fancy_df = filtered_comparison_df[cols] 

In [36]:
fancy_df

,fStrategies,CropUtil,Best Strategy Predicted,Predicted CropUtil
0,0,0.022530,64,0.662942
2,8,0.636089,128,0.739873
3,8,0.634186,16384,0.709342
4,0,0.768649,256,0.769121
5,128,0.413703,256,0.606014
...,...,...,...,...
1790931,0,0.262695,4,0.702255
1790932,4,0.640358,2048,0.811541
1790933,8,0.617906,64,0.819938
1790934,8,0.509921,64,0.739062


In [88]:
difference_crop_util = filtered_comparison_df['Predicted CropUtil'] - filtered_comparison_df['CropUtil']

In [89]:
difference_crop_util

0          0.640412
2          0.103784
3          0.075157
4          0.000472
5          0.192311
             ...   
1790931    0.439560
1790932    0.171183
1790933    0.202032
1790934    0.229141
1790935    0.177583
Length: 1534589, dtype: float64

In [91]:
difference_crop_util.mean()

0.16398244124083172